In [1]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Input, concatenate


In [2]:
train = pd.read_csv('../data/RFMiD_Training_Labels.csv').set_index('ID')
test = pd.read_csv('../data/RFMiD_Testing_Labels.csv').set_index('ID')
eval = pd.read_csv('../data/RFMiD_Validation_Labels.csv').set_index('ID')


In [3]:
train.columns


Index(['Disease_Risk', 'DR', 'ARMD', 'MH', 'DN', 'MYA', 'BRVO', 'TSLN', 'ERM',
       'LS', 'MS', 'CSR', 'ODC', 'CRVO', 'TV', 'AH', 'ODP', 'ODE', 'ST',
       'AION', 'PT', 'RT', 'RS', 'CRS', 'EDN', 'RPEC', 'MHL', 'RP', 'CWS',
       'CB', 'ODPM', 'PRH', 'MNF', 'HR', 'CRAO', 'TD', 'CME', 'PTCR', 'CF',
       'VH', 'MCA', 'VS', 'BRAO', 'PLQ', 'HPED', 'CL'],
      dtype='object')

In [4]:
X_train = train.drop(columns='Disease_Risk')
y_train = train['Disease_Risk']
X_eval  = eval.drop(columns='Disease_Risk')
y_eval = eval['Disease_Risk']


In [5]:
X_train.sample()


,DR,ARMD,MH,DN,MYA,BRVO,TSLN,ERM,LS,MS,...,CME,PTCR,CF,VH,MCA,VS,BRAO,PLQ,HPED,CL
ID,,,,,,,,,,,,,,,,,,,,,
335,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
def load_and_preprocess_image(image_id, image_folder, target_size=(224, 224)):
    """
    Load and preprocess an image given its ID.
    """
    image_path = os.path.join(image_folder, f'{image_id}.png')
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image)
    image = image / 255.0
    return image


In [7]:
image_folder = '../data/training_images'
images = np.array([load_and_preprocess_image(row_id, image_folder) for row_id in X_train.index])


In [8]:
eval_image_folder = '../data/eval_images'
eval_images = np.array([load_and_preprocess_image(row_id, image_folder) for row_id in X_eval.index])


In [9]:
image_input = Input(shape=(224, 224, 3))
x = Conv2D(32, (3, 3), activation='relu')(image_input)
x = MaxPooling2D(2, 2)(x)
x = Flatten()(x)


#  NOTE: row_input would cause data leakage so it's been commented out.

# row_input = Input(shape=(X_train.shape[1],))
# y = Dense(64, activation='relu')(row_input)

# combined = concatenate([x, y])

z = Dense(12, activation='relu')(x)
z = Dense(64, activation='relu')(z)
z = Dense(1, activation='sigmoid')(z)


model = Model(inputs=image_input, outputs=z)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [10]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 394272)            0         
                                                                 
 dense (Dense)               (None, 12)                4731276   
                                                                 
 dense_1 (Dense)             (None, 64)                832       
                                                             

In [11]:
model.fit(
    images, y_train,
    validation_data=(eval_images, y_eval),
    epochs=10,
    batch_size=32
)


Epoch 1/10
60/60 [==============================] - 13s 203ms/step - loss: 0.5317 - accuracy: 0.7688 - val_loss: 0.5628 - val_accuracy: 0.7906
Epoch 2/10
60/60 [==============================] - 12s 193ms/step - loss: 0.4585 - accuracy: 0.8083 - val_loss: 0.5840 - val_accuracy: 0.7906
Epoch 3/10
60/60 [==============================] - 12s 191ms/step - loss: 0.4301 - accuracy: 0.8203 - val_loss: 0.6531 - val_accuracy: 0.7906
Epoch 4/10
60/60 [==============================] - 14s 226ms/step - loss: 0.4320 - accuracy: 0.8177 - val_loss: 0.8325 - val_accuracy: 0.7906
Epoch 5/10
60/60 [==============================] - 13s 214ms/step - loss: 0.4125 - accuracy: 0.8240 - val_loss: 0.6672 - val_accuracy: 0.7906
Epoch 6/10
60/60 [==============================] - 12s 198ms/step - loss: 0.4115 - accuracy: 0.8313 - val_loss: 0.5341 - val_accuracy: 0.7906
Epoch 7/10
60/60 [==============================] - 13s 208ms/step - loss: 0.4166 - accuracy: 0.8286 - val_loss: 0.5651 - val_accuracy: 0.7906

In [12]:
model.evaluate(eval_images, y_eval)


20/20 [==============================] - 1s 54ms/step - loss: 0.6031 - accuracy: 0.7656


[0.6031017303466797, 0.765625]